### This is Owen Monroe's Homework Assignment # 3 for Text Mining IS 567
October 8, 2023

1. Train and evaluate a unigram-based baseline classifier

In [23]:
import pandas as pd
import numpy as np
import nltk
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [24]:
# Loading Data and Removing Empties

train_df = pd.read_csv('chatgpt_train.csv')
test_df = pd.read_csv('chatgpt_test.csv')

full_train_df = train_df.dropna()
full_test_df = test_df.dropna()

print(len(full_train_df['review'].to_list()))
print(len(full_test_df['review'].to_list()))

full_train_df


1829
458


,date,title,review,rating
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4
...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1
1830,5/19/2023 1:23,Goated app,Best app,5
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1


In [25]:
# Function to Convert Ratings to Three-Way Ratings

def change_five_to_three(rating):
    if rating in [1, 2]:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    elif rating in [4, 5]:
        return 'positive'
    else:
        return 'unknown'

In [26]:
# New Rating List for Train and Test Data

three_rate_train_list = full_train_df['rating'].apply(change_five_to_three).to_list()
three_rate_test_list = full_test_df['rating'].apply(change_five_to_three).to_list()


full_train_df['three rating'] = three_rate_train_list
full_test_df['three rating'] = three_rate_test_list


/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_48066/896341391.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_train_df['three rating'] = three_rate_train_list


In [27]:
full_train_df

,date,title,review,rating,three rating
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4,positive
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4,positive
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4,positive
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4,positive
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4,positive
...,...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1,negative
1830,5/19/2023 1:23,Goated app,Best app,5,positive
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1,negative
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1,negative


In [28]:
full_test_df

,date,title,review,rating,three rating
0,5/19/2023 6:09,error unsupported country,cant login,2,negative
1,5/19/2023 9:39,Hype junk,More harm than help.,1,negative
2,5/19/2023 4:12,your gpt 4 is fake,Fix it,1,negative
3,5/20/2023 3:01,Please impose IPadOS,We need IPadOS!!!,5,positive
4,5/19/2023 20:49,Amazing,Great,5,positive
...,...,...,...,...,...
453,5/19/2023 0:17,Andrew Justino Wilson 5/19/23,This has to be a beginning to something crazy ...,5,positive
454,5/18/2023 19:13,Superb AI,I?€?ve been using chat and have been a proud p...,5,positive
455,5/18/2023 18:27,Fantastic App with Room for Enhancements,The ChatGPT iOS app is an outstanding product....,5,positive
456,5/18/2023 17:17,"Awesome technology, deplorable tactics",Sam Altman?€?s blatant attempt at regulatory c...,2,negative


In [29]:
train_rev_text = full_train_df["review"]
test_rev_text = full_test_df["review"]

# Setting n-gram range to uni-grams
vectorizer = CountVectorizer(ngram_range = (1, 1))

# Creating training data representation
train_rev_uni_cv = vectorizer.fit_transform(train_rev_text.values.astype('U'))
print(train_rev_uni_cv.shape,"\n") 

# Creating test data representation
test_rev_uni_cv = vectorizer.transform(test_rev_text.values.astype('U'))
print(test_rev_uni_cv.shape,"\n") 

print(train_rev_uni_cv)

(1829, 5551) 

(458, 5551) 

  (0, 5224)	1
  (0, 4985)	9
  (0, 4932)	5
  (0, 3613)	1
  (0, 5282)	1
  (0, 3134)	1
  (0, 493)	2
  (0, 5253)	2
  (0, 776)	2
  (0, 3331)	3
  (0, 3165)	2
  (0, 5454)	1
  (0, 1302)	3
  (0, 2139)	1
  (0, 806)	1
  (0, 5421)	1
  (0, 2666)	6
  (0, 1426)	1
  (0, 4267)	3
  (0, 3883)	2
  (0, 3184)	1
  (0, 2660)	3
  (0, 3692)	2
  (0, 1352)	1
  (0, 4906)	16
  :	:
  (1824, 2949)	1
  (1824, 3593)	1
  (1824, 2644)	1
  (1824, 5301)	1
  (1825, 291)	1
  (1825, 527)	1
  (1826, 2666)	1
  (1826, 2456)	1
  (1826, 413)	1
  (1826, 3247)	1
  (1826, 5433)	1
  (1826, 1695)	1
  (1827, 2666)	1
  (1827, 474)	1
  (1827, 5464)	1
  (1827, 2619)	1
  (1827, 3372)	1
  (1827, 1976)	1
  (1827, 2807)	1
  (1827, 794)	1
  (1827, 195)	1
  (1827, 3699)	1
  (1827, 2328)	1
  (1828, 2666)	1
  (1828, 2135)	1


In [31]:
# Define true labels from train set

x_train_cv = train_rev_uni_cv
y_train_rev_series = full_train_df["rating"]
x_test_cv = test_rev_uni_cv
y_test_rev_series = full_test_df["rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv, y_train_rev_series)

# Predict the labels for the test data

predictions = mnb_model.predict(x_test_cv)
predictions

array([1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 1, 5, 5, 5, 1, 1, 5, 5, 5,
       1, 5, 5, 1, 5, 5, 1, 5, 4, 5, 5, 5, 1, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 1, 5, 3, 4, 5, 5, 1, 5, 5, 1, 1, 5,
       5, 5, 5, 1, 5, 4, 5, 1, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 5, 5, 1, 5, 5, 5, 1, 1, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5,
       1, 1, 1, 5, 1, 1, 5, 5, 5, 5, 5, 2, 2, 5, 3, 1, 5, 5, 5, 1, 5, 5,
       5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 2, 1, 1, 5, 5, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5,
       5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 2, 5, 5, 1, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 1, 3, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5, 5, 5, 5,
       1, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 1, 5,